<a href="https://colab.research.google.com/github/DanielEstrada971102/CUFICO-2018-2/blob/master/Copia_de_HistogrammingInROOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# With this notebook we will:
## 1. Install packages needed for root
## 2. Download a binary root that compiled in google drive "colaboratory" environement
## 3. Modify the PATH in order to tell python where to look for pyROOT
## 4. Finally, test ROOT installation with a simple example

##### Author: Jose Ruiz
##### Maintainer: Jose Ruiz
##### Developed at: Universidad de Antioquia


### 1. Install packages needed for root

In [0]:
!apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2).
libxext-dev is already the newest version (2:1.3.3-1).
libxext-dev set to manually installed.
libxft-dev is already the newest version (2.3.2-1).
binutils is already the newest version (2.30-21ubuntu1~18.04).
binutils set to manually installed.
dpkg-dev is already the newest version (1.19.0.5ubuntu2.1).
dpkg-dev set to manually installed.
g++ is already the newest version (4:7.3.0-3ubuntu2.1).
g++ set to manually installed.
gcc is already the newest version (4:7.3.0-3ubuntu2.1).
gcc set to manually installed.
gfortran is already the newest version (4:7.3.0-3ubuntu2.1).
git is already the newest version (1:2.17.1-1ubuntu0.4).
libx11-dev is already the newest version (2:1.6.4-3ubuntu0.1).
libx11-dev set to manually installed.
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:


### 2. Mount the drive to get the root binary and untar it 

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/Team Drives/PyROOT_in_Google/"

In [0]:
!tar -xf /content/drive/Team\ Drives/PyROOT_in_Google/root_build_google_yikes.tar


### 3. Modify the PATH in order to tell python where to look for pyROOT


In [0]:
import sys
sys.path.append("/content/root_build/")
sys.path.append("/content/root_build/bin/")
sys.path.append("/content/root_build/include/")
sys.path.append("/content/root_build/lib/")

In [0]:
from ROOT import TCanvas, TH1F, TF1, TGraph
import numpy as np
from glob import glob
from time import sleep

In [0]:
# Se cargan los datos como una matriz de 12*20 
filenames = sorted(glob('drive/My Drive/Datos_TComputo/Tiempo_Computo_*.txt'))

Datos = []

for i in filenames:
  Datos.append(np.loadtxt(fname = i, delimiter = ','))
Datos = np.array(Datos)

In [0]:
# Se encuentra el maximo tiempo de computo para cada n

T_max = []
for i in range(0, 12):
  T_max.append(max(Datos[i]))
T_max = np.array(T_max)

In [0]:
# Se hacen 12 histogramas con los datos cargados de tiempos de computo
mean_gauss = []
for i in range(0, 12):
  c = TCanvas('c', 'canvas', 600, 700)
  gaus = TF1('f', 'gaus', min(Datos[i]),max(Datos[i])) # esta funcion se usara para hacer un ajuste gaussiano
  histograma = TH1F('histograma', 'Tiempo De computo para n = %d'%(i*5), 20, min(Datos[i]),max(Datos[i]))
  
  for j in range(0,20):
    histograma.Fill(Datos[i][j]) # se agregan los datos de tiempo de computo al histograma
  
  histograma.SetLineColor(2) 
  histograma.Draw()
  
  histograma.Fit('f') # se hace un fit del histograma con 'f' que corresponde a la funcion gaussiana
  histograma.SetLineColor(3)
  histograma.Draw('C')
  
  mean_gauss.append(gaus.GetParameter(1)) # se almacena el dato de fiteo que corresponde a la media de la distribucion
  # gaussiana(representa el valor medio de estabilizacion del sistema), este se usara luego para graficas estos valores para cada n.
  
  c.SetGrid()
  c.Update()
  c.SaveAs("drive/My Drive/Imag_Parcial/Hist_gauss%d.png"%(i))
  
mean_gauss = np.array(mean_gauss)

In [0]:
# Se define una funcion que apartir de dos arreglos de datos hace una grafica y un fit de estos y los guarda como archivo png, esta se usa para fitear 
# los tiempos maximos de computo y los valores medios de las guassianas.
def Graph_and_Fit_arreglo(size, x, y, title, xAxis, yAxis, fit, nameImage):
  c = TCanvas('c', 'title', 700, 600)

  #x = np.array([5,10,15,20,25,30,35,40,45,50,55,60])
  g = TGraph(size)
  g.SetTitle(title)

  for i in range(0,size):
    g.SetPoint(i, x[i], y[i])

  func = TF1('e', fit, min(x), max(x)) 
  g.Fit('e')

  g.SetMarkerStyle(9)
  c.SetGrid()
  ejex = g.GetXaxis()
  ejex.SetTitle(xAxis)
  ejey = g.GetYaxis()
  ejey.SetTitle(yAxis)
  g.Draw('AP')
  c.SaveAs('drive/My Drive/Imag_Parcial/' + nameImage + '.png')
  

In [90]:
# Se hace un fit para los datos de tiempos maximos de computo
x = np.array([5,10,15,20,25,30,35,40,45,50,55,60])
Graph_and_Fit_arreglo(12, x, T_max, 'Tiempos maximos de computo vs n', 'n', 'Tiempo maximo de computo', '[0] + [1]*exp([2]*x)', 'fit_Tiempos_maximos')


****************************************
Minimizer is Minuit / Migrad
Chi2                      =      0.13776
NDf                       =            9
Edm                       =  1.66116e-06
NCalls                    =          464
p0                        =    0.0234284   +/-   0.0642968   
p1                        =    0.0012266   +/-   0.00315579  
p2                        =     0.110222   +/-   0.0432127   


Info in <TCanvas::Print>: png file drive/My Drive/Imag_Parcial/fit_Tiempos_maximos.png has been created


La función que ajusta los datos de máximo tiempo de computo es :
$ T_{max}(n)\approx 0.023 + 0.0012 e^{0.11n}$

In [92]:
# Se hace un fit para los datos de valor medio de estabilizacion del sistema  
Graph_and_Fit_arreglo(12, x, mean_gauss, 'Valor medio de estabilizacion vs n', 'n', 'valor medio de estabilizacion', '[0] + [1]*x + [2]* x**2', 'fit_valMed_estabilizacion')


****************************************
Minimizer is Minuit / Migrad
Chi2                      =      13.2657
NDf                       =            9
Edm                       =  2.86334e-21
NCalls                    =           67
p0                        =     -1.32704   +/-   1.25478     
p1                        =     0.158932   +/-   0.0887573   
p2                        =  -0.00337299   +/-   0.00132928  


Info in <TCanvas::Print>: png file drive/My Drive/Imag_Parcial/fit_valMed_estabilizacion.png has been created
